In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_excel("датасет 15072022.xlsx")
df.head()

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ
2,2112 Продукция,8415900009; 8415900009; 8415900009; 8415900009...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Клапаны противопожарные нормально открытые, кл...",Клапаны противопожарные марки КПВ.01(EI60) пря...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu
3,50677 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы крышные радиальные дымоудаления с ...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu
4,51009 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы радиальные дымоудаления типов ВР-8...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu


In [12]:
df_test1 = df[["Группа продукции", "Технические регламенты", "Коды ТН ВЭД ЕАЭС", "Общее наименование продукции"]].copy()
df_test1.head()

,Группа продукции,Технические регламенты,Коды ТН ВЭД ЕАЭС,Общее наименование продукции
0,Узлы пересечения противопожарных преград кабел...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,4016999708,Проходки кабельные модульные универсальные огн...
1,"Противопожарные окна, двери, двери шахт лифтов...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,7308909809,Ворота противопожарные металлические распашные...
2,"Клапаны противопожарные нормально открытые, кл...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8415900009; 8415900009; 8415900009; 8415900009...,Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы крышные радиальные дымоудаления с ...
4,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы радиальные дымоудаления типов ВР-8...


In [21]:
with open("ТН ВЭД\TNVED\TNVED3.Txt", encoding='cp866') as fout_tnved3:
    dict_common = dict()
    fout_tnved3.readline()
    for line in fout_tnved3.readlines():
        line = line.replace('\xa0', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_common[ls[0]] = ls[2]
    for i in dict_common.items():
        print(i, "\n")

('01', 'ЖИВЫЕ ЖИВОТНЫЕ ПРОЧИЕ') 

('02', 'МЯСО И ПИЩЕВЫЕ МЯСНЫЕ СУБПРОДУКТЫ, СОЛЕНЫЕ, В РАССОЛЕ, СУШЕНЫЕ ИЛИ КОПЧЕНЫЕ; ПИЩЕВАЯ МУКА ТОНКОГО И ГРУБОГО ПОМОЛА ИЗ МЯСА ИЛИ МЯСНЫХ СУБПРОДУКТОВ') 

('03', 'МУКА ТОНКОГО И ГРУБОГО ПОМОЛА И ГРАНУЛЫ ИЗ РЫБЫ, РАКООБРАЗНЫХ, МОЛЛЮСКОВ И ПРОЧИХ ВОДНЫХ БЕСПОЗВОНОЧНЫХ, ПРИГОДНЫЕ ДЛЯ УПОТРЕБЛЕНИЯ В ПИЩУ') 

('04', 'НАСЕКОМЫЕ И ПРОЧИЕ ПИЩЕВЫЕ ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ') 

('05', 'ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ; ПАВШИЕ ЖИВОТНЫЕ ГРУППЫ 01 ИЛИ 03, НЕПРИГОДНЫЕ ДЛЯ УПОТРЕБЛЕНИЯ В ПИЩУ') 

('06', 'ЛИСТЬЯ, ВЕТКИ И ДРУГИЕ ЧАСТИ РАСТЕНИЙ БЕЗ ЦВЕТКОВ ИЛИ БУТОНОВ, ТРАВЫ, МХИ И ЛИШАЙНИКИ, ПРИГОДНЫЕ ДЛЯ СОСТАВЛЕНИЯ БУКЕТОВ ИЛИ ДЛЯ ДЕКОРАТИВНЫХ ЦЕЛЕЙ, СВЕЖИЕ, ЗАСУШЕННЫЕ, ОКРАШЕННЫЕ, ОТБЕЛЕННЫЕ, ПРОПИТАННЫЕ ИЛИ ПОДГОТОВЛЕННЫЕ ДРУГИМИ СПОСОБАМИ') 

('07', 'МАНИОК, МАРАНТА, САЛЕП, ЗЕМЛЯНАЯ ГРУША, ИЛИ ТОПИНАМБУР, СЛАДКИЙ КАРТОФЕЛЬ, ИЛИ БАТАТ, И АНАЛОГИЧНЫЕ КОРН

In [20]:
with open("ТН ВЭД\TNVED\TNVED2.Txt", encoding='cp866') as fout_tnved2:
    dict_group = dict()
    fout_tnved2.readline()
    for line in fout_tnved2.readlines():
        line = line.replace('\xa0', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_group[ls[1]] = ls[2]
    for i in dict_group.items():
        print(i, "\n")

('01', 'ЖИВЫЕ ЖИВОТНЫЕ') 

('02', 'МЯСО И ПИЩЕВЫЕ МЯСНЫЕ СУБПРОДУКТЫ') 

('03', 'РЫБА И РАКООБРАЗНЫЕ, МОЛЛЮСКИ И ПРОЧИЕ ВОДНЫЕ БЕСПОЗВОНОЧНЫЕ') 

('04', 'МОЛОЧНАЯ ПРОДУКЦИЯ; ЯЙЦА ПТИЦ; МЕД НАТУРАЛЬНЫЙ; ПИЩЕВЫЕ ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ') 

('05', 'ПРОДУКТЫ ЖИВОТНОГО ПРОИСХОЖДЕНИЯ, В ДРУГОМ МЕСТЕ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ') 

('06', 'ЖИВЫЕ ДЕРЕВЬЯ И ДРУГИЕ РАСТЕНИЯ; ЛУКОВИЦЫ, КОРНИ И ПРОЧИЕ АНАЛОГИЧНЫЕ ЧАСТИ РАСТЕНИЙ; СРЕЗАННЫЕ ЦВЕТЫ И ДЕКОРАТИВНАЯ ЗЕЛЕНЬ') 

('07', 'ОВОЩИ И НЕКОТОРЫЕ СЪЕДОБНЫЕ КОРНЕПЛОДЫ И КЛУБНЕПЛОДЫ') 

('08', 'СЪЕДОБНЫЕ ФРУКТЫ И ОРЕХИ; КОЖУРА ЦИТРУСОВЫХ ПЛОДОВ ИЛИ КОРКИ ДЫНЬ') 

('09', 'КОФЕ, ЧАЙ, МАТЕ, ИЛИ ПАРАГВАЙСКИЙ ЧАЙ, И ПРЯНОСТИ') 

('10', 'ЗЛАКИ') 

('11', 'ПРОДУКЦИЯ МУКОМОЛЬНО-КРУПЯНОЙ ПРОМЫШЛЕННОСТИ; СОЛОД; КРАХМАЛЫ; ИНУЛИН; ПШЕНИЧНАЯ КЛЕЙКОВИНА') 

('12', 'МАСЛИЧНЫЕ СЕМЕНА И ПЛОДЫ; ПРОЧИЕ СЕМЕНА, ПЛОДЫ И ЗЕРНО; ЛЕКАРСТВЕННЫЕ РАСТЕНИЯ И РАСТЕНИЯ ДЛЯ ТЕХНИЧЕСКИХ ЦЕЛЕЙ; СОЛОМА И ФУРАЖ') 

('1